In [ ]:
import os
os.chdir("/home/mbikandi/Documents/s3ts/")

In [ ]:
from storage.har_datasets import STSDataset, StreamingTimeSeries, StreamingTimeSeriesCopy

In [ ]:
from s3ts.api.ucr import load_ucr_classification
from s3ts.api.ts2sts import finite_random_STS
import numpy as np
import torch

X, Y, mapping = load_ucr_classification("BasicMotions")
#X, Y, mapping = load_ucr_classification("GunPoint")
print(X.shape, Y.shape, len(np.unique(Y)))

STS, SCS = finite_random_STS(X, Y, length=60)
print(STS.shape, SCS.shape)

In [ ]:
ds = StreamingTimeSeries(STS, SCS, wsize=64, wstride=1)

In [ ]:
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader

class LSTSDataset(LightningDataModule):

    """ Data module for the experiments. """

    STS: np.ndarray     # data stream
    SCS: np.ndarray     # class stream
    DM: np.ndarray      # dissimilarity matrix

    data_split: dict[str: np.ndarray]    
                        # train / val / test split
    batch_size: int     # dataloader batch size

    def __init__(self,
            stsds: StreamingTimeSeries,    
            data_split: dict, batch_size: int, 
            random_seed: int = 42, 
            num_workers: int = 1
            ) -> None:

        # save parameters as attributes
        super().__init__()
        
        self.batch_size = batch_size
        self.random_seed = random_seed
        self.num_workers = num_workers

        self.stsds = stsds
        self.wdw_len = self.stsds.wsize
        self.wdw_str = self.stsds.wstride
        self.sts_str = False

        # gather dataset info   
        self.n_dims = self.stsds.STS.shape[1]
        self.n_classes = len(np.unique(self.stsds.SCS))

        # convert to tensors
        if not torch.is_tensor(self.stsds.STS):
            self.stsds.STS = torch.from_numpy(self.stsds.STS).to(torch.float32)
        if not torch.is_tensor(self.stsds.SCS):
            self.stsds.SCS = torch.from_numpy(self.stsds.SCS).to(torch.int64)

        train_indices = self.stsds.indices[data_split["train"](self.stsds.indices)]
        test_indices = self.stsds.indices[data_split["test"](self.stsds.indices)]
        val_indices = self.stsds.indices[data_split["val"](self.stsds.indices)]

        self.ds_train = StreamingTimeSeriesCopy(self.stsds, train_indices)
        self.ds_test = StreamingTimeSeriesCopy(self.stsds, test_indices)
        self.ds_val = StreamingTimeSeriesCopy(self.stsds, val_indices)
        
    def train_dataloader(self) -> DataLoader:
        """ Returns the training DataLoader. """
        return DataLoader(self.ds_train, batch_size=self.batch_size, 
            num_workers=self.num_workers, shuffle=True,
            pin_memory=True, persistent_workers=True)

    def val_dataloader(self) -> DataLoader:
        """ Returns the validation DataLoader. """
        return DataLoader(self.ds_val, batch_size=self.batch_size, 
            num_workers=self.num_workers, shuffle=False,
            pin_memory=True, persistent_workers=True)

    def test_dataloader(self) -> DataLoader:
        """ Returns the test DataLoader. """
        return DataLoader(self.ds_test, batch_size=self.batch_size, 
            num_workers=self.num_workers, shuffle=False,
            pin_memory=True, persistent_workers=True)
    
    def predict_dataloader(self) -> DataLoader:
        """ Returns the test DataLoader. """
        return DataLoader(self.ds_test, batch_size=self.batch_size, 
            num_workers=self.num_workers, shuffle=False,
            pin_memory=True, persistent_workers=True)

In [ ]:
indices_shuffled = np.arange(ds.indices.shape[0])
np.random.shuffle(indices_shuffled)

data_split = {
    "train": lambda x: np.isin(x, indices_shuffled[:5000]),
    "val": lambda x: np.isin(x, indices_shuffled[5000:5200]),
    "test": lambda x: np.isin(x, indices_shuffled[5200:]),
}

dm = LSTSDataset(ds, data_split=data_split, batch_size=16, random_seed=42, num_workers=8)

In [ ]:
@torch.jit.script
def dtw_compute(dist_tensor: torch.Tensor, grad_tensor: torch.Tensor, w: float) -> tuple[torch.Tensor, torch.Tensor]:
    for i in range(1, dist_tensor.shape[2]):
        for j in range(1, dist_tensor.shape[3]):
            # elements has shape (n, k, 3)
            elements = torch.stack([w * dist_tensor[:, :, i, j-1], dist_tensor[:, :, i-1, j], w * dist_tensor[:, :, i-1, j-1]], dim=2)

            value, id = torch.min(elements, dim=2) # shape (n, k)

            dist_tensor[:,:, i, j] += value

            grad_tensor[id==0][:, :, i, j] += w * grad_tensor[id==0][:, :, i, j-1]

    return dist_tensor, grad_tensor

def torch_dtw_fast(x, y, w):
    # shape of x (n, dim, x_len) y (m, dim, y_len)    
    # performs convolution-like operation, for each kernel the DF
    # (of shape (kernel_size, T)) is computed, then summed across channels
    # x has shape (batch, c, time_dimension)

    # compute pairwise diffs (squared)
    p_diff = x[:,None,:,None,:] - y[None,:,:,:,None] # shape (n, n_kernel, d, Kernel_size, T)
    euc_d = torch.square(p_diff).sum(2) # shape (n, n_kernel, kernel_size, T)

    # compute dtw
    DTW = euc_d.clone()
    DTW[:,:,0,:] = torch.cumsum(DTW[:,:,0,:], dim=2)
    DTW[:,:,:,0] = torch.cumsum(DTW[:,:,:,0], dim=2)

    # p_diff contains the partial derivatives of DTW[n, k, i, j] wrt K[k, d, i] (dims (n, k, d, i, j))
    p_diff = p_diff / euc_d[:,:, None, :, :]

    DTW, p_diff = dtw_compute(DTW, p_diff, w)

    return DTW, p_diff

class torch_dtw(torch.autograd.Function):

    @staticmethod
    def forward(x, y, w):
        DTW, p_diff = torch_dtw_fast(x, y, w)
        return DTW, p_diff
    
    @staticmethod
    def setup_context(ctx, inputs, output):
        DTW, p_diff = output
        ctx.save_for_backward(p_diff)
    
    @staticmethod
    def backward(ctx, dtw_grad, p_diff_grad):
        p_diff, = ctx.saved_tensors
        mult = (p_diff * dtw_grad[:,:,None,:,:])
        return mult.mean(dim=(1, 3)), mult.mean(dim=(0, 4)), None

In [ ]:
class DTWLayer(torch.nn.Module):
    def __init__(self, n_patts, d_patts, l_patts, rho: float = 1) -> None:
        super().__init__()

        self.w: torch.float32 = rho ** (1/l_patts)
        self.patts = torch.nn.Parameter(torch.randn(n_patts, d_patts, l_patts))
    
    def forward(self, x):
        return torch_dtw.apply(x, self.patts, self.w)[0][:,:,:,32:]

In [ ]:
model = torch.nn.Sequential(
    DTWLayer(4, 6, 32, rho=0.1),
    torch.nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2),
    torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2),
    torch.nn.Flatten(),
    torch.nn.Linear(in_features=25*32, out_features=4),
)

In [ ]:
optimizer = torch.optim.Adam([{"params": model[1:].parameters(), "lr": 0.001},
                              {"params": model[0].parameters(), "lr": 0.001}])
criterion = torch.nn.functional.cross_entropy

for i in range(10):
    running_loss = 0
    for id, batch in enumerate(dm.train_dataloader()):
        data = batch["series"]
        cl = batch["label"][:, -1]

        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, cl)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(running_loss/id)